# Test the model
Run the appropriate cell below to test YAML or Ansible generation

### Test YAML

In [1]:
checkpoint = "santacoder-finetuned-the-stack-yaml"
test_input = "- hello: world"

### Test Ansible

In [2]:
#checkpoint = "santacoder-finetuned-the-stack-ansible-ec2"
#test_input = "- name: start an instance type g5.xlarge"

In [3]:
!pip install transformers torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 60.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 152.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.4/346.4 kB 277.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 285.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 773.4/773.4 kB 280.4 MB/s eta 0:00:00

[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer

pretrained = "bigcode/santacoder"
device = "cuda" # for GPU usage or "cpu" for CPU usage

In [5]:
import torch
import os

print("PyTorch version:", torch.__version__)

# Check if CUDA is available
if torch.cuda.is_available():
    print("CUDA is available.")
    print("Number of GPU:", torch.cuda.device_count())
    for i in range(torch.cuda.device_count()):
        print(f"Current Device : {torch.cuda.current_device}")
        print(f"Device {i}: {torch.cuda.get_device_name(i)}")
        print(torch.cuda.get_device_properties(i))
        print(f"Memory GB: {torch.cuda.get_device_properties(i).total_memory / 1024**3:.2f} GB")
        print(f"GPU Allocated: {torch.cuda.memory_allocated(i) / 1024 ** 3:.2f} GB")
        print(f"GPU Cached:    {torch.cuda.memory_reserved(i) / 1024 ** 3:.2f} GB")
        
        
else:
    print("CUDA is not available. No GPU detected.")


print(torch.cuda._get_device_index)
print(torch.cuda.current_device)

#torch.cuda.set_device(3)


CUDA_VISIBLE_DEVICES = os.environ.get('CUDA_VISIBLE_DEVICES')
print(CUDA_VISIBLE_DEVICES)

PyTorch version: 2.0.1+cu118
CUDA is available.
Number of GPU: 1
Current Device : <function current_device at 0x7f5285002d30>
Device 0: NVIDIA A10G
_CudaDeviceProperties(name='NVIDIA A10G', major=8, minor=6, total_memory=22502MB, multi_processor_count=80)
Memory GB: 21.98 GB
GPU Allocated: 0.00 GB
GPU Cached:    0.00 GB
<function _get_device_index at 0x7f528522a280>
<function current_device at 0x7f5285002d30>
2


In [6]:
!nvidia-smi

Fri Mar  8 09:28:15 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.14              Driver Version: 550.54.14      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A10G                    On  |   00000000:00:1B.0 Off |                    0 |
|  0%   27C    P0             66W /  300W |   21346MiB /  23028MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [7]:
import os
from huggingface_hub import login
# Note: Prefer notebook_login() but this was not prompting properly
login(token=os.environ.get('HUGGINGFACE_TOKEN'))
#print(token)

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /opt/app-root/src/.cache/huggingface/token
Login successful


In [8]:
tokenizer = AutoTokenizer.from_pretrained(pretrained, trust_remote_code=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
model = AutoModelForCausalLM.from_pretrained(checkpoint, trust_remote_code=True).to(device)

In [10]:
tokenizer.pad_token = tokenizer.eos_token
encoding = tokenizer.encode_plus(
    test_input,
    add_special_tokens=True,
    padding='longest',
    max_length=128,
    truncation=True,
    return_tensors='pt'
).to(device)
input_ids = encoding['input_ids']
attention_mask = encoding['attention_mask']
output_ids = model.generate(
    input_ids=input_ids,
    attention_mask=attention_mask,
    max_length=128,
    num_return_sequences=1,
    pad_token_id=tokenizer.eos_token_id
)
print(tokenizer.decode(output_ids[0], skip_special_tokens=True))

- hello: world
  - name: User-Agent
    value:
      - AZURECLI/2.28.1 azsdk-python-azure-mgmt-network/23.0.0 Python/3.8.9 (Windows-10-10.0.19041-SP0)
    headers:
      User-Agent: AZURECLI/2.28.1 azsdk-python-azure-mgmt-network/23.0.0 Python/3.8.9 (Windows-10-10.0.19041
